# Factor Analysis for Face Recognition

Assign a label indicating which of $M$ possible identities a face belongs to, based on a vector of grayscale pixel intensities $x$. Model the likelihood for each class using a factor analyzer:

$$
    \text{Pr}(x \ | \ w_i = k) = \text{Norm}_{x}(\mu_k, \phi_k\phi_k^T + \Sigma_k)\\[0.7em]
$$
Learn the parameters for the $k$th identity using the images of faces corresponding to that identity. Use expecation maximization to learn these parameters. Assign priors $\text{Pr}(w = k)$ according to each face's prevalence in the database. To evaluate a new face image $x_i$, compute the posterior $\text{Pr}(w_i|x_i)$.

In [1]:
# Data:            Continuous - vectors of grayscale pixel intensities.
# World states:    Discrete - categories corresponding to face identities.
# Model:           Generative - factor analyzers each parameterized by unique means, factors, and diagonal covariance matrices.
# Learning:        Expectation maximization of each model's parameters against images of their corresponding face IDs.
# Inference:       Computation of the posterior over face IDs, given an input face image vector.

# Requirements:
#             -> [st] training_data[]    - Array with rows of grayscale face images 
#             -> [st] training_labels[]  - Vector of the faces' corresponding IDs
#             -> [fn] dnorm()            - Multivariate normal density function
#             -> [fn] posterior()        - Receives image vector, returns class probabilities
#             -> [st] priors[]           - Class priors indexed by world state
#             -> [fn] fit_priors()       - Receives training set, returns vector of class priors
#             -> [fn] likelihood()       - Receives image vector + world state, returns probability
#             -> [fn] fit_likelihoods()  - Iterates over world states and fits their corresponding likelihood functions
#             -> [fn] EM()               - Maximizes likelihood for fit_likelihoods()
#             -> [fn] E_step()           - Computes hidden variable posteriors for EM()
#             -> [fn] M_step()           - Maximizes boundary w.r.t. likelihood fn parameters for EM()
#             -> [fn] boundary()         - Computes boundary value for EM()
#             -> [st] likelihood_params[]- List containing parameters corresponding to kth likelihood function
#             -> [st] test_data[]        - Vector of test image grayscale pixel intensities

#` 1. Import and flatten the data.
#` 2. Fit the prior distribution     : prior = fit_prior(vector of class labels)
#`    1.1. Compute and return the relative class sizes.
#` 3. Fit the likelihood distribution: likelihood_parameters = fit_likelihood(matrix of image vectors, vector of class labels, n_factors)
#`   3.1. Iterate over each class and maximize its images' likelihoods - Pr(x|w,th) - wr.t. th via EM.
#        -> Call: [mu_k, phi_k, covar_k] = EM(matrix of class image vectors, n_factors)
#       3.1.1. Randomly initialize vector mu_k, matrix phi_k, diag. matrix covar_k.
#       3.1.2. E_step(): Maximize the boundary w.r.t. the density functions over the hidden variable.
#       3.1.3. M_step(): Maximize the boundary's value w.r.t. mu_k, phi_k, and covar_k.
#       3.1.4. boundary(): Compute the boundary's value given the parameters and density functions.
#`    3.2. Store [mu_k, phi_k, covar_k] in an array such that they are indexed by world state.
#`    3.3. Return the array of parameter values.
#  4. Inference: [vector of class probabilities] = posterior(input_vec, parameter_arr)
#    4.1. Compute the product likelihood(input_vec, parameter_arr, k)*prior[k] for all world states k.
#       4.1.1. likelihood() is a multivariate normal density calculation.
#    4.2. Normalize and return this vector of unnormalized probabilities.
# 5. Display the image along with the computed class probabilities.

# What do the factors represent? Directions in which covariance is greatest among pixels.
# *Extracting the factors and plotting them as images would be interesting

In [2]:
import glob
import pytest
import numpy as np
from scipy.misc import imread, imresize

In [3]:
# Import and label the training data
face_ids        = [fp[-6:] for fp in glob.iglob('.\\data\\face-identification\\select-faces\\*')]
face_fps        = glob.glob('.\\data\\face-identification\\select-faces\\*\\*.jpg')
training_data   = np.array([np.ravel(imresize(imread(fp, flatten=True), size = [60, 60])) for fp in face_fps])
training_labels = [face_ids.index(name) for name in [fp.split('\\')[-2] for fp in face_fps]]

In [4]:
# Test functions for fit_likelihood()
def test_likelihood_inputs(training_data, training_labels, n_factors):
    assert n_factors > 0
    assert type(training_labels)    == list
    assert type(training_data)      == np.ndarray
    assert type(training_labels[0]) == int
    assert len(training_labels)     == training_data.shape[0]

def test_likelihood_outputs(klikelihood_params, datum_length, n_factors):
    assert test_mean_dim(klikelihood_params[1], datum_length)
    assert test_factor_dim(klikelihood_params[2], datum_length, n_factors)
    assert test_covar_dim(klikelihood_params[3], datum_length)
    assert test_covar_psd(klikelihood_params[3])
    assert test_covar_sym(klikelihood_params[3])

def test_likelihood_mean_dim(mean_vector, datum_length):
    assert len(mean_vector) == datum_length

def test_likelihood_factor_dim(factor_matrix, datum_length, n_factors):
    assert factor_matrix.shape == (datum_length, n_factors)

def test_likelihood_covar_positive(covar_matrix):
    assert np.all(covar_matrix >= 0)
    
def test_likelihood_covar_dim(covar_matrix, datum_length):
    assert covar_matrix.shape == (datum_length, datum_length)
    
def test_likelihood_covar_sym(covar_matrix):
    assert np.all(covar_matrix == covar_matrix.T)

def test_likelihood_covar_psd(covar_matrix):
    assert np.all(np.linalg.eigvals(covar_matrix) >= 0)

In [5]:
def fit_prior(training_labels):
    # Receives a list of training labels indicating face ID
    # Returns a numpy array of ID prior probabilities
    priors = [sum(training_labels == ID)/len(training_labels) for ID in np.unique(training_labels)]
    return np.array(priors)

In [15]:
def fit_likelihood(training_data, training_labels, n_factors):
    # training_data   -> numpy array with rows of grayscale image vectors
    # training_labels -> list of associated integer face IDs
    # n_factors       -> Number of factors to use in factor analyzer
    # Iterates over world states and fits the parameters of their associated likelihood distributions
    test_likelihood_inputs(training_data, training_labels, n_factors)
    class_IDs         = np.unique(training_labels)
    likelihood_params = [None]*len(class_IDs) # List containing a set of parameters for each world state
    datum_length      = training_data.shape[1]
    for k in np.unique(class_IDs):
        k_data               = training_data[training_labels == k, :] # Images containing kth person
        klikelihood_params   = EM(k_data, n_factors, datum_length) # Fit the kth FA's parameters -> [mu_k, phi_k, covar_k]
        assert test_likelihood_outputs(klikelihood_params, datum_length, n_factors)
        likelihood_params[k] = klikelihood_params
    return likelihood_params

In [19]:
def EM(training_data, n_factors, datum_length):
    # Returns parameters that fit a factor analyzer to the given data
    # training_data -> np array with rows of vectorized input images
    # n_factors     -> Number of factors to model the data's covariance with
    # 1. Randomly initialize parameters
    mu_upd      = np.random.uniform([255]*datum_length) # Vector
    phi_upd     = np.array([np.random.normal(size = datum_length) for _ in range(n_factors)]) # Matrix
    covar_upd   = np.random.normal()*np.identity(datum_length)
    n_datum = training_data.shape[0]
    # 2. Iterate over E and M steps until boundary ceases to shift
    error = 1
    while (error > 1e-4) :
        [mu, phi, covar]             = [mu_upd, phi_upd, covar_upd]
        expectations                 = E_step(training_data, mu, phi, covar, n_datum) # [N x 2] np.array: [(E[h_i], E[h_i h_i.T]), ...]
        [mu_upd, phi_upd, covar_upd] = M_step(training_data, expectations, n_datum) #
        error                        = np.max(abs(np.array([mu_upd - mu], [phi_upd - phi], [covar_upd - covar])))
    return [mu_upd, phi_upd, covar_upd]
# The hidden variable is a vector describing the shift in the normal's mean through the subspace - this shift is conditional
# on each image, meaning that well-represented images will produce many shifts to a particular part of the subspace. This
# further means that the factors will tend to be aligned in the direction of shift (see the computation of the factors in
# the M-step).

In [10]:
def E_step(training_data, mu, phi, covar, n_datum):
    mu = np.mat(mu).T; phi = np.mat(phi); covar = np.mat(covar); training_data = np.mat(training_data);
    I  = np.identity(len(mu))
    expectations = [None]*n_datum
    for i in range(n_datum):
        x                = np.mat(training_data[i, :]).T
        inv_term         = np.linalg.inv(phi.T * np.linalg.inv(covar) * phi + I)
        E_h              = inv_term * phi.T * np.linalg.inv(covar) * (x - mu)
        E_hh             = inv_term + E_h * E_h.T
        expectations[i]  = [E_h, E_hh]
    return expectations

In [11]:
def M_step(training_data, expectations, n_datum):
    x         = training_data
    # Mu
    mu_hat    = np.sum(training_data, axis = 0)
    # Phi
    list_E_hh = np.array([expec[2] for expec in expectations])
    list_E_h  = np.array([expec[1] for expec in expectations])
    t1_list   = [x[i, :].T * list_E_h[i].T for i in range(n_datum)] # List of matrices
    phi_t1    = np.sum(t1_list, axis = 2)
    phi_t2    = np.linalg.inv(np.sum(t2_list, axis = 2))
    phi_hat   = phi_t1 * phi_t2
    # Sigma
    for i in range(n_datum):
        sum_of_sq += np.diag((x[i, :].T - mu_hat) * (x[i, :].T - mu_hat).T - phi_hat * list_E_h[i] * (x[i, :].T - mu_hat).T)
    covar     = sum_of_sq/n_datum
    return [mu_hat, phi_hat, covar_hat]

In [20]:
n_factors = 1
fit_parameters = fit_likelihood(training_data, training_labels, n_factors)

ValueError: shapes (3600,1) and (3600,3600) not aligned: 1 (dim 1) != 3600 (dim 0)

In [ ]:
^ DEAL WITH THIS WHEN YOU GET BACK